In [113]:
# !pip install langchain_chroma
# ! pip install sentence_transformers
# ! pip install faiss-cpu
! pip install -q groq



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


A RAG system is a system that help our llm to generalize in a specifique content or local content(private).
building a rag system require the steps:

1. devide the document into chunks to suit the llm context window
2. embedd the chunks and then store them in a vector db
3. after the query arrive, embedd it then retreive k-chunks that has similarity with the query
4. finaly include the k-chunks into the prompt

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from groq import Groq
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

c:\Users\dell\Desktop\MS_DS\M1\S2\AppAutomaique\Projet\loi001_project\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# 1. Preparing the data, pdf file

In [2]:
loader = PyPDFLoader("./loi-n-01-00-portant-organisation-de-lenseignement-supérieur.pdf")
pages = loader.load_and_split()

In [5]:
pages[10].page_content

"9 \nIl veille, sous la supervision du président de l'université, au re spect de la législation et de la réglementation en vigueur et  du \nrèglement intérieur dans l'enceinte de l'établissement et pe ut prendre toutes les mesures que les circonstances exigent \nconformément à la législation en vigueur. \n \nArticle 22 \nLe conseil de l'établissement comprend des membres de droit, des représentants élus des personnels enseignants et des \npersonnels administratifs et techniques,  des représentants élus des étudiants, ainsi que des membres désignés parmi des \npersonnalités extérieures. \nLa composition des conseils des établissements, le mode de désignation ou d'élection de leurs membres, ainsi que les \nmodalités de leur fonctionnement sont fixés par voie réglementaire. \nOutre les attributions qui lui sont dévolues par la présente loi, le conseil de l'établissement: \n9 connaît de toutes les questions relatives aux missions et à la bonne marche de l'établissement et peut formuler des

the second part is to devide the text to chunks, we will use the langchaine text splitter for this

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
splits = text_splitter.split_documents(pages)

In [7]:
type(splits[1])

langchain_core.documents.base.Document

In [55]:
len(splits)

109

As the embedding models from langchain require api_key, which may cost me something, i will do an alternative which is using a free embedding model from hugginface as they are open source

because the suitable format for huggin face model are plaint text, i transformed the splited text which is of type -langchain_core.documents.base.Document-, to simple text chuncks

In [8]:
text_chunks = [doc.page_content for doc in splits]

In [9]:
text_chunks[0]

'1'

# 2. Embedding generation and store them to the vector DB

for this task i will be using Aleph Alpha's semantic embeddings from the langchain embedding models 

In [10]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
embeddings = [model.encode(chunk) for chunk in text_chunks]

In [13]:
embedding_array = np.array(embeddings) #to store it into faiss

In [14]:
embedding_array.shape

(109, 384)

Store the embedding into a vector DB using FAISS

In [15]:
embedding_dim = embedding_array.shape[1]  # Dimension of your embeddings
index = faiss.IndexFlatL2(embedding_dim)

In [16]:
index.add(embedding_array)

## 3. Retrieving the chuncks with most similarity

In [17]:
def retrieving_chunks(query):
    # Example query
    # query = "Quelles sont les prestations et le financement des services sociaux destinés aux étudiants dans le cadre de la vie universitaire ?"
    query_embedding = model.encode(query)  # Generate embedding for the query

    # Convert to NumPy array and reshape for FAISS
    query_embedding = np.array([query_embedding], dtype=np.float32)

    # Perform the search
    k = 3  # Number of nearest neighbors to retrieve
    distances, indices = index.search(query_embedding, k)
    matching_chunks = [text_chunks[i] for i in indices[0]]
    return '/n'.join(matching_chunks)

## 3. Generating Step

In [18]:
with open('config.json') as config_file:
    config = json.load(config_file)
    
api_key = config['api_key']

In [19]:
client = Groq(
    api_key=api_key,
)

In [20]:
def llm_generation(question, context):
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": "repondre au question en se basant sur le context donnée"
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": "question : "+question+",context: "+context
            }
        ],
        model="llama3-70b-8192",
    )

    print(chat_completion.choices[0].message.content)

In [21]:
ret = retrieving_chunks(query= 'donner moi article 75')

In [22]:
respoons = llm_generation('donner moi article 75',ret)

Article 75 : Dans les cinq ans suivant le prononcé d'une décision de condamnation, une personne qui a été condamnée pour une infraction prévue aux articles 63 et 64 et au présent article, et qui commet une infraction de même nature, sera considérée comme récidiviste.
